In [3]:
import pandas as pd
import pathlib
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Find latest output folder
output_dir = pathlib.Path("../output")
subdirs = [d for d in output_dir.iterdir() if d.is_dir()]
subdirs.sort()
if not subdirs:
    raise FileNotFoundError("No output dirs found")
latest_run = subdirs[-1]

print(f"Loading data from: {latest_run}")

# Load dataframes
df_market = pd.read_csv(latest_run / "sim_history_market.csv")
df_demand = pd.read_csv(latest_run / "sim_history_demand.csv")
df_company = pd.read_csv(latest_run / "sim_history_company.csv")

# Merge for combined analysis
df = pd.merge(df_company, df_market, on="month", how="left")
df = pd.merge(df, df_demand, on="month", how="left")

# Rename for convenience/compat
if "demand_north_sea" in df.columns:
    df.rename(columns={"demand_north_sea": "demand_ns"}, inplace=True)

print(f"Merged data: {len(df)} rows")


Loading data from: ../output/2025-12-15T12-54-25
Merged data: 9 rows


In [4]:

# Ensure types
df["month"] = df["month"].astype(int)
# Numeric conversion for safekeeping
numeric_cols = ["oil_price", "cash_musd", "steel_price", "steel_demand"]
demand_cols = [c for c in df.columns if c.startswith("demand_")]
for c in numeric_cols + demand_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Derived metrics
df["rigs_total"] = df["rigs_active"] + df["rigs_warm"] + df["rigs_cold"]
df["utilisation"] = np.where(df["rigs_total"] > 0, df["rigs_active"] / df["rigs_total"], np.nan)

# Market table (unique per month)
market_cols = ["month"] + numeric_cols + demand_cols
market = (
    df[[c for c in market_cols if c in df.columns]]
    .drop_duplicates()
    .sort_values("month")
)

# Latest snapshot per company
latest = df.sort_values("month").groupby("company").tail(1).sort_values("cash_musd", ascending=False)

# ================
# CHARTS
# ================

# 1) Oil price over time
fig = px.line(market, x="month", y="oil_price", title="Oil price over time")
fig.show()

# 2) Demand over time (all regions)
m2 = market.melt(id_vars=["month"], value_vars=demand_cols, var_name="region", value_name="demand")
fig = px.line(m2, x="month", y="demand", color="region", title="Regional demand over time")
fig.show()

# 3) Company cash over time
fig = px.line(df, x="month", y="cash_musd", color="company", title="Cash over time (by company)")
fig.show()

# 4) Cash indexed to 100 at first month (growth view)
base = df.sort_values("month").groupby("company").first().reset_index()[["company", "cash_musd"]].rename(columns={"cash_musd": "cash0"})
df_idx = df.merge(base, on="company", how="left")
df_idx["cash_index"] = 100 * df_idx["cash_musd"] / df_idx["cash0"]
fig = px.line(df_idx, x="month", y="cash_index", color="company", title="Cash indexed (start = 100)")
fig.show()

# 5) Utilisation over time
fig = px.line(df, x="month", y="utilisation", color="company", title="Utilisation over time (active / total rigs)")
fig.update_yaxes(tickformat=".0%")
fig.show()

# 6) Fleet state counts over time (stacked area per company)
long_rigs = df.melt(
    id_vars=["month", "company"],
    value_vars=["rigs_active", "rigs_warm", "rigs_cold"],
    var_name="state",
    value_name="count",
)
fig = px.area(
    long_rigs,
    x="month",
    y="count",
    color="state",
    facet_row="company",
    title="Fleet state over time (stacked, per company)",
)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

# 7) End-of-sim cash bar chart
fig = px.bar(latest, x="company", y="cash_musd", title="End-of-sim cash ($m) by company")
fig.show()

# 8) Scatter: oil price vs demand (North Sea)
if "demand_ns" in market.columns:
    fig = px.scatter(market, x="oil_price", y="demand_ns", title="Oil price vs Demand (North Sea)", trendline="ols")
    fig.show()
if "demand_gom" in market.columns:
    fig = px.scatter(market, x="oil_price", y="demand_gom", title="Oil price vs Demand (GOM)", trendline="ols")
    fig.show()

# 9) Scatter: demand vs cash (per company)
if "demand_gom" in df.columns:
    fig = px.scatter(
        df,
        x="demand_gom",
        y="cash_musd",
        color="company",
        title="Cash vs Demand (GOM) by company",
    )
    fig.show()

# 10) Heatmap: cash by company & month
pivot_cash = df.pivot_table(index="company", columns="month", values="cash_musd", aggfunc="mean")
fig = px.imshow(
    pivot_cash,
    aspect="auto",
    title="Heatmap: cash ($m) by company (rows) and month (columns)",
)
fig.show()

# 11) Rolling volatility of oil (3-month rolling std)
market2 = market.copy()
market2["oil_roll_std_3"] = market2["oil_price"].rolling(3).std()
fig = px.line(market2, x="month", y="oil_roll_std_3", title="Oil price volatility (3-month rolling std)")
fig.show()

# 12) Rolling cash change (delta) per company
df_delta = df.sort_values(["company", "month"]).copy()
df_delta["cash_delta"] = df_delta.groupby("company")["cash_musd"].diff()
fig = px.line(df_delta, x="month", y="cash_delta", color="company", title="Monthly cash change ($m)")
fig.show()

# 13) Distribution: monthly cash change
fig = px.histogram(df_delta.dropna(subset=["cash_delta"]), x="cash_delta", color="company", barmode="overlay",
                   title="Distribution of monthly cash changes ($m)")
fig.show()

# 14) Rank over time: who’s leading in cash?
ranks = df.copy()
ranks["rank"] = ranks.groupby("month")["cash_musd"].rank(ascending=False, method="dense")
fig = px.line(ranks, x="month", y="rank", color="company", title="Cash rank over time (1 = richest)")
fig.update_yaxes(autorange="reversed", dtick=1)
fig.show()

print("✅ Loaded rows:", len(df))
print("✅ Companies:", df['company'].nunique())
print("✅ Months:", df['month'].nunique())


✅ Loaded rows: 9
✅ Companies: 3
✅ Months: 3
